In [ ]:
import os
import sys
import copy
import time
import random
from datetime import timedelta, datetime

import numpy as np
import polars as pl
import plotly.graph_objects as go

from tqdm.notebook import tqdm
from IPython.display import display
from ipywidgets.widgets import HBox

import torch
from torch import nn, optim, cuda
from torch.utils.data import DataLoader, Dataset

In [2]:
ROOT_PATH = './'
DRIVE_PATH = 'Colab/RecSys-TP'

# When on Colab, use Google Drive as the root path to persist and load data
if 'google.colab' in sys.modules:
    from google.colab import drive, output

    output.enable_custom_widget_manager()

    drive.mount('/content/drive')
    ROOT_PATH = os.path.join('/content/drive/My Drive/', DRIVE_PATH)
    os.makedirs(ROOT_PATH, exist_ok=True)
    os.chdir(ROOT_PATH)

In [3]:

RANDOM_SEED = 1984

BATCH_SIZE = 1024
K_UI = 64
K_IL = 64

TOTAL_EPOCHS = 50

BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-8
AMSGRAD = False
WEIGHT_DECAY = 0.01

WARMUP_RATIO = 0.05
# LEARNING_RATE = 0.04
# USE_SCHEDULER = True

LEARNING_RATE = 0.005
USE_SCHEDULER = False

EVAL_K = 10

PYTORCH_DEVICE = 'cpu'

# Use NVIDIA GPU if available
if cuda.is_available():
    PYTORCH_DEVICE = 'cuda'

# Use Apple Metal backend if available
if torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("Your device supports MPS but it is not installed. Checkout https://developer.apple.com/metal/pytorch/")
    else:
        PYTORCH_DEVICE = 'mps'

print(f"Using {PYTORCH_DEVICE} device for PyTorch")

Using cuda device for PyTorch


In [4]:

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.mps.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
# Carrega os dados do dataset (http://ocelma.net/MusicRecommendationDataset/lastfm-1K.html)
user_profiles = pl.read_csv("./data/lastfm-dataset-1K/userid-profile.tsv", separator="\t")
user_interactions = pl.read_csv("./data/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv",
                                separator="\t", has_header=False, quote_char=None)

# Renomeia as colunas
user_profiles.columns = ["user_id", "gender", "age", "country", "registered"]
user_interactions.columns = ["user_id", "timestamp", "artist_id", "artist_name", "track_id", "track_name"]

# Descarta linhas com valores nulos nas iterações
user_interactions = user_interactions.drop_nulls()

display(user_profiles.sample(10, seed=RANDOM_SEED))
display(user_interactions.sample(10, seed=RANDOM_SEED))

user_id,gender,age,country,registered
str,str,i64,str,str
"""user_000809""","""m""",null,"""Finland""","""Jun 8, 2005"""
"""user_000112""","""f""",30,"""Turkey""","""Mar 25, 2006"""
"""user_000086""","""f""",27,null,"""Sep 21, 2007"""
"""user_000403""","""f""",null,"""United States""","""May 17, 2006"""
"""user_000863""",null,null,"""United Kingdom""","""Oct 15, 2004"""
"""user_000720""","""f""",null,"""Norway""","""Jun 29, 2007"""
"""user_000985""","""f""",null,"""Australia""","""May 22, 2006"""
"""user_000487""","""m""",null,"""Netherlands""","""Mar 8, 2006"""
"""user_000049""",null,null,null,"""Jan 11, 2006"""


user_id,timestamp,artist_id,artist_name,track_id,track_name
str,str,str,str,str,str
"""user_000806""","""2008-08-09T19:28:14Z""","""fc61dd75-880b-44ba-9ba9-c7b643…","""Prefuse 73""","""60f3a1c9-e756-4a48-8a3e-c44140…","""Altoid Addiction (Interlude)"""
"""user_000108""","""2007-12-19T17:14:42Z""","""6ae51665-8261-4ae5-883f-189965…","""Filter""","""784f24f6-6fa7-44e7-81a5-a7b592…","""The Best Things"""
"""user_000079""","""2008-12-01T00:09:19Z""","""48896dee-a985-424d-9849-84802f…","""Johnny Mathis""","""e77a742f-eb2c-417c-9b48-45e404…","""Can'T Get Out Of This Mood"""
"""user_000407""","""2008-05-20T15:37:40Z""","""8bfac288-ccc5-448d-9573-c33ea2…","""Red Hot Chili Peppers""","""7a3e8796-a0b3-4999-b268-4e2d47…","""Otherside"""
"""user_000861""","""2008-02-12T22:25:17Z""","""31aa6f87-8d00-4ae9-a5cc-6d7eee…","""Alphaville""","""f11939cf-9ad1-45b8-b927-a89b00…","""Big In Japan"""
"""user_000728""","""2009-05-21T19:04:24Z""","""41c86965-305a-482d-bc1e-2daeca…","""Skankfunk""","""e48c9ed7-34ac-44aa-ab5a-3d792a…","""Melo-Pole"""
"""user_000990""","""2009-03-31T10:20:28Z""","""1bc69a93-8020-4e07-8b05-0b6331…","""The Cliks""","""f89ba590-0226-4c65-b5c5-4b69ee…","""Complicated"""
"""user_000174""","""2005-05-03T18:42:12Z""","""fc178247-53b6-4702-ad77-546cb0…","""The Exposures""","""b222a53c-3168-43e5-a40d-65e95a…","""Sake Rock"""
"""user_000412""","""2005-11-04T22:08:08Z""","""86e736b4-93e2-40ff-9e1c-fb7c63…","""Barenaked Ladies""","""05b34070-535a-4b92-aa9b-ecb97c…","""Call And Answer"""


In [6]:
# Cria um mapeamento dos IDs para números
item_ids = user_interactions['track_id'].unique().to_list()
user_ids = user_interactions['user_id'].unique().to_list()

item_id_index = {id: i + 1 for i, id in enumerate(item_ids)}
item_id_index_rev = {v: k for k, v in item_id_index.items()}

user_id_index = {id: i + 1 for i, id in enumerate(user_ids)}
user_id_index_rev = {v: k for k, v in user_id_index.items()}

# Aplica as transformações no dataframe
dataset = user_interactions.select(
    pl.col('user_id').replace_strict(user_id_index).alias('uid'),
    pl.col('track_id').replace_strict(item_id_index).alias('iid'),
    pl.col('timestamp').cast(pl.Datetime).alias('ts')
).sort('uid', 'ts')

max_uid = dataset['uid'].max()
max_iid = dataset['iid'].max()

display(dataset.head(10))

uid,iid,ts
i64,i64,datetime[μs]
1,570357,2007-01-23 23:42:33
1,783400,2007-09-29 21:23:06
2,853295,2008-10-26 19:02:07
2,908835,2008-10-26 19:05:03
2,630316,2008-10-26 19:06:35
2,807123,2008-10-26 19:10:18
2,102945,2008-10-26 19:15:14
2,627368,2008-10-26 19:17:24
2,846988,2008-10-26 19:20:19


In [7]:
threshold = timedelta(minutes=30)

# Marca cada música como se ela representa ou não o início de uma nova sessão
new_session_col = dataset.group_by('uid') \
    .agg(
    # Separa por usuário (uma sessão é de um usuário)
    pl.col('iid'),

    # Computa a diferença entre a música atual e a anterior, se essa 
    # diferença for nula ou maior igual ao nosso limite de tempo,
    # então é um início de sessão.
    (pl.col('ts').diff().fill_null(threshold) >= threshold).alias('start'),
    # Expande as duas listas simultaneamente para que voltemos ao formato inicial
).explode('iid', 'start')['start']

# Agora, para criar um id para cada sessão, vamos usar a função cum_sum (isso 
# funciona pois nossos dados estão ordenados por usuário e timestamp, 
# respectivamente)
dataset_with_session = dataset.with_columns(new_session_col.cum_sum().alias('sid'))

display(dataset_with_session)

uid,iid,ts,sid
i64,i64,datetime[μs],u32
1,570357,2007-01-23 23:42:33,1
1,783400,2007-09-29 21:23:06,2
2,853295,2008-10-26 19:02:07,3
2,908835,2008-10-26 19:05:03,3
2,630316,2008-10-26 19:06:35,3
…,…,…,…
992,761668,2009-02-25 03:11:06,899896
992,39072,2009-02-25 03:24:43,899896
992,684383,2009-04-26 18:19:39,899897


In [8]:
# Agrupa as reproduções por sessão em um array
dataset_grouped = dataset_with_session.group_by('sid', 'uid').agg(pl.col('iid').alias('iids'))
display(dataset_grouped.head())

# Imprime a distribuição do tamanho das sessões
display(dataset_grouped['iids'].list.len().describe())

# Reduz o número de amostras para reduzir o tempo de treinamento (ao custo de
# uma redução na qualidade da base de dados) Seguiremos com 100k sessões das
# ~900k presentes na base.
dataset_grouped = dataset_grouped.sample(1e5, shuffle=True)

sid,uid,iids
u32,i64,list[i64]
17069,20,"[416165, 692840, 443780]"
306335,340,"[528791, 761245, … 31978]"
462690,515,[652788]
764787,857,"[389473, 102561, … 78345]"
814871,918,"[45780, 243086, … 610623]"


statistic,value
str,f64
"""count""",899898.0
"""null_count""",0.0
"""mean""",18.871339
"""std""",43.066011
"""min""",1.0
"""25%""",3.0
"""50%""",9.0
"""75%""",21.0
"""max""",5435.0


In [9]:
def gen_negative(iid: int) -> int:
    global max_iid
    while True:
        negative = np.random.randint(1, max_iid)
        if negative != iid:
            return negative
        
# Para o FPMC, precisamos de tuplas do tipo (usuário, item anterior, próximo item real, próximo item negativo [gerado no dataset])
def gen_data(uid: int, iids: list[int]) -> list[tuple[int, int, int]]:
    data = []
    for prev, nxt in zip(iids[:-1], iids[1:]):
        data.append((uid, prev, nxt))
    return data

def train_slice(uid: int, data: list[int]) -> list[tuple[int, int, int]]:
    if len(data) < 3:
        return gen_data(uid, data)

    return gen_data(uid, data[:-2])


def validation_slice(uid: int, data: list[int]) -> list[tuple[int, int, int]]:
    if len(data) < 3:
        return []

    return gen_data(uid, data[-3:-1])


def test_slice(uid: int, data: list[int]) -> list[tuple[int, int, int]]:
    if len(data) < 3:
        return []

    return gen_data(uid, data[-2:])

def flatten(data: list[list[tuple[int, int, int]]]) -> list[tuple[int, int, int]]:
    return [x for sublist in data for x in sublist]

train_data: list[tuple[int, int, int]] = flatten([train_slice(uid, data.to_list()) for uid, data in
                                                     zip(dataset_grouped['uid'], dataset_grouped['iids'])])
validation_data: list[tuple[int, int, int]] = flatten([validation_slice(uid, data.to_list()) for uid, data in
                                                                 zip(dataset_grouped['uid'], dataset_grouped['iids']) if len(data) > 2])
test_data: list[tuple[int, int, int]] = flatten([validation_slice(uid, data.to_list()) for uid, data in
                                                                      zip(dataset_grouped['uid'], dataset_grouped['iids']) if len(data) > 2])

In [ ]:
sample_negatives = {}
# Utilizado para avaliação/teste
def get_or_generate_negatives(iid: int, sample_size: int = 100) -> list[int]:
    global sample_negatives
    if iid not in sample_negatives:
        sample_negatives[iid] = np.random.randint(1, max_iid, size=sample_size)
    return sample_negatives[iid]

class FPMCDataset(Dataset):
    def __init__(self, data: list[tuple[int, int, int]]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return *self.data[index], gen_negative(self.data[index][2])

train_dataloader = DataLoader(FPMCDataset(train_data), batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
validation_dataloader = DataLoader(FPMCDataset(validation_data), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_dataloader = DataLoader(FPMCDataset(test_data), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [ ]:
class FPMC(nn.Module):
    def __init__(self, num_users, num_items, k_UI=64, k_IL=64):
        """
        Initializes the FPMC model.

        Args:
            num_users (int): The number of users in the dataset.
            num_items (int): The number of items in the dataset.
            k_UI (int, optional): The embedding dimension for the user embeddings. Defaults to 64.
            k_IL (int, optional): The embedding dimension for the item embeddings. Defaults to 64.
        """
        super(FPMC, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.k_UI = k_UI
        self.k_IL = k_IL
        self.sqrt_kUI = k_UI ** 0.5
        self.sqrt_kIL = k_IL ** 0.5
        

        self.IL = nn.Embedding(self.num_items, self.k_IL)
        self.LI = nn.Embedding(self.num_items, self.k_IL)

        self.UI = nn.Embedding(self.num_users, self.k_UI)
        self.IU = nn.Embedding(self.num_items, self.k_UI)

    def forward(self, uid, prev_iid, next_iid):
        # O modelo é composto por dois componentes: MF e FMC
        x_mf = torch.sum(self.UI(uid) * self.IU(next_iid), dim=1) / self.sqrt_kUI
        x_fmc = torch.sum(self.IL(next_iid) * self.LI(prev_iid), dim=1) / self.sqrt_kIL
        return x_mf + x_fmc


class BPRLoss(nn.Module):
    def __init__(self, gamma=1e-10):
        super(BPRLoss, self).__init__()
        self.gamma = gamma

    def forward(self, pos_score: torch.Tensor, neg_score: torch.Tensor):
        loss = -torch.log(self.gamma + torch.sigmoid(pos_score - neg_score)).mean()
        return loss


In [12]:


def evaluate_model(
        model: FPMC,
        loader: DataLoader,
        device: str = PYTORCH_DEVICE,
) -> tuple[float, float]:
    global EVAL_K
    ndcg = 0
    hit = 0
    total = 0

    model.eval()
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating Validation: ", total=len(loader), leave=False):
            uids, prev_iids, next_iids, _ = batch
            for i in range(len(batch)):
                total += 1
                
                uid = uids[i].repeat(101)
                prev_iid = prev_iids[i].repeat(101)
                negative_iids = get_or_generate_negatives(next_iids[i])
                next_iid = torch.tensor([next_iids[i], *negative_iids])
            
                outputs = model(uid.to(device), prev_iid.to(device), next_iid.to(device))
               
                ranking = outputs.argsort(descending=True)
                
                for rank, item in enumerate(ranking[:EVAL_K]):
                    if item == 0:
                        ndcg += 1 / np.log2(rank + 2)
                        hit += 1
        
    return ndcg / total, hit / total

def train_model(
        model: FPMC,
        optimizer: optim.Optimizer,
        scheduler: optim.lr_scheduler.LRScheduler | None,
        train_loader: DataLoader,
        val_loader: DataLoader,
        num_epochs: int,
        device: str = PYTORCH_DEVICE,
) -> tuple[list[float], list[float], list[float], tuple[dict, dict, dict], tuple[dict, dict, dict]]:
    best_ndcg = 0
    best_hit = 0
    best_ndcg_epoch = 0
    best_hit_epoch = 0

    losses = []
    ndcgs = []
    hits = []
    lrs = []

    best_ncdg_model_state = None
    best_ncdg_optimizer_state = None
    best_ncdg_scheduler_state = None

    best_hit_model_state = None
    best_hit_optimizer_state = None
    best_hit_scheduler_state = None

    # Plot the loss and other metrics

    fig_loss_widget = go.FigureWidget(layout=go.Layout(title="Loss"))
    fig_ndcg_widget = go.FigureWidget(layout=go.Layout(title="NDCG@" + str(EVAL_K)))
    fig_hit_widget = go.FigureWidget(layout=go.Layout(title="HIT@" + str(EVAL_K)))
    fig_lr_widget = go.FigureWidget(layout=go.Layout(title="Learning Rate"))


    fig_loss_widget.add_scatter(x=np.arange(len(losses)) + 1, y=losses)
    fig_ndcg_widget.add_scatter(x=np.arange(len(ndcgs)) + 1, y=ndcgs)
    fig_hit_widget.add_scatter(x=np.arange(len(hits)) + 1, y=hits)
    fig_lr_widget.add_scatter(x=np.arange(len(lrs)) + 1, y=lrs)

    fig_loss_widget.update_xaxes(title_text='Epoch')
    fig_ndcg_widget.update_xaxes(title_text='Epoch')
    fig_hit_widget.update_xaxes(title_text='Epoch')
    fig_lr_widget.update_xaxes(title_text='Epoch')

    fig_loss_widget.update_yaxes(title_text='Loss', type='log')
    fig_ndcg_widget.update_yaxes(title_text='NDCG@' + str(EVAL_K))
    fig_hit_widget.update_yaxes(title_text='Epoch@' + str(EVAL_K))
    fig_lr_widget.update_yaxes(title_text='Learning Rate')

    display(HBox([fig_loss_widget, fig_lr_widget]))
    display(HBox([fig_ndcg_widget, fig_hit_widget]))

    # Wait for widgets to load
    time.sleep(1)

    criterion = BPRLoss()
    
    steps = 0
    for epoch in tqdm(range(num_epochs), desc="Epoch: "):
        model.train()
        epoch_loss = 0

        lrs.append([pg['lr'] for pg in optimizer.param_groups][0])

        for batch in tqdm(train_loader, desc="Training: ", total=len(train_loader), leave=False):
            model.zero_grad()
            
            uid, prev_iid, next_iid, negative_iid = batch
            
            uid = uid.to(device)
            prev_iid = prev_iid.to(device)
            next_iid = next_iid.to(device)
            negative_iid = negative_iid.to(device)
            
            positive_out = model(uid, prev_iid, next_iid)
            negative_out = model(uid, prev_iid, negative_iid)

            loss = criterion(positive_out, negative_out)

            loss.backward()
            epoch_loss += loss.item()
            
            optimizer.step()

            if scheduler is not None:
                scheduler.step()

            steps += 1

        ndcg, hit = evaluate_model(model, val_loader, device=device)

        if ndcg > best_ndcg:
            best_ndcg = ndcg
            best_ndcg_epoch = epoch
            best_ncdg_model_state = copy.deepcopy(model.state_dict())
            best_ncdg_optimizer_state = copy.deepcopy(optimizer.state_dict())
            if scheduler is not None:
                best_ncdg_scheduler_state = copy.deepcopy(scheduler.state_dict())

        if hit > best_hit:
            best_hit = hit
            best_hit_epoch = epoch
            best_hit_model_state = copy.deepcopy(model.state_dict())
            best_hit_optimizer_state = copy.deepcopy(optimizer.state_dict())
            if scheduler is not None:
                best_hit_scheduler_state = copy.deepcopy(scheduler.state_dict())

        losses.append(epoch_loss)
        ndcgs.append(ndcg)
        hits.append(hit)


        fig_loss_widget.data[0].x = np.arange(len(losses)) + 1
        fig_loss_widget.data[0].y = losses
        fig_ndcg_widget.data[0].x = np.arange(len(ndcgs)) + 1
        fig_ndcg_widget.data[0].y = ndcgs
        fig_hit_widget.data[0].x = np.arange(len(hits)) + 1
        fig_hit_widget.data[0].y = hits
        fig_lr_widget.data[0].x = np.arange(len(lrs)) + 1
        fig_lr_widget.data[0].y = lrs

    print(f"Best NDCG@{EVAL_K} Epoch: {best_ndcg_epoch + 1}, NDCG@{EVAL_K}: {best_ndcg:.4f}")
    print(f"Best HIT@{EVAL_K} Epoch: {best_hit_epoch + 1}, HIT@{EVAL_K}: {best_hit:.4f}")

    return losses, ndcgs, hits, \
        (best_ncdg_model_state, best_ncdg_optimizer_state, best_ncdg_scheduler_state), \
        (best_hit_model_state, best_hit_optimizer_state, best_hit_scheduler_state)


In [13]:
model = FPMC(num_users=max_uid + 1, num_items=max_iid + 1, k_UI=K_UI, k_IL=K_IL)

model.to(PYTORCH_DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, betas=(BETA_1, BETA_2), eps=EPS, weight_decay=WEIGHT_DECAY, amsgrad=AMSGRAD)

scheduler = None
if USE_SCHEDULER:
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=LEARNING_RATE,
        total_steps=TOTAL_EPOCHS * len(train_dataloader),
        pct_start=WARMUP_RATIO,
        anneal_strategy="linear",
    )

results = train_model(model, optimizer, scheduler, train_dataloader, validation_dataloader, TOTAL_EPOCHS, device=PYTORCH_DEVICE)

    'data': [{'type': 'scatter', 'uid': '60aec7db-1be0-4cd9-9e42-008034e08e35', …

    'data': [{'type': 'scatter', 'uid': '03867d58-e459-4b36-9a1d-68c8a2fd1595', …

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/1595 [00:00<?, ?it/s]

Evaluating Validation:   0%|          | 0/79 [00:00<?, ?it/s]

...

Training:   0%|          | 0/1595 [00:00<?, ?it/s]

Evaluating Validation:   0%|          | 0/79 [00:00<?, ?it/s]

Training:   0%|          | 0/1595 [00:00<?, ?it/s]

Evaluating Validation:   0%|          | 0/79 [00:00<?, ?it/s]

Best NDCG@10 Epoch: 44, NDCG@10: 0.6980
Best HIT@10 Epoch: 46, HIT@10: 0.7975


In [14]:
losses, ndcgs, hits, \
    (best_ncdg_model_state, best_ncdg_optimizer_state, best_ncdg_scheduler_state), \
    (best_hit_model_state, best_hit_optimizer_state, best_hit_scheduler_state) = results

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(f"models/fpmc/{timestamp}/", exist_ok=True)

torch.save(best_ncdg_model_state, f"models/fpmc/{timestamp}/best_ncdg_model_state.pt")
torch.save(best_hit_model_state, f"models/fpmc/{timestamp}/best_hit_model_state.pt")

torch.save(best_ncdg_optimizer_state, f"models/fpmc/{timestamp}/best_ncdg_optimizer_state.pt")
torch.save(best_hit_optimizer_state, f"models/fpmc/{timestamp}/best_hit_optimizer_state.pt")

torch.save(best_ncdg_scheduler_state, f"models/fpmc/{timestamp}/best_ncdg_scheduler_state.pt")
torch.save(best_hit_scheduler_state, f"models/fpmc/{timestamp}/best_hit_scheduler_state.pt")

In [15]:
fig = go.Figure(layout = go.Layout(title="Loss"))
fig.add_scatter(x=np.arange(len(losses)) + 1, y=losses)
fig.update_xaxes(title_text='Epoch', type='log')
fig.update_yaxes(title_text='Loss')
display(fig)

fig = go.Figure(layout = go.Layout(title="NDCG@" + str(EVAL_K)))
fig.add_scatter(x=np.arange(len(ndcgs)) + 1, y=ndcgs)
fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='NDCG@' + str(EVAL_K))
display(fig)

fig = go.Figure(layout = go.Layout(title="HIT@" + str(EVAL_K)))
fig.add_scatter(x=np.arange(len(hits)) + 1, y=hits)
fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Epoch@' + str(EVAL_K))
display(fig)

In [18]:
# Load the best model during training
test_model = FPMC(num_users=max_uid + 1, num_items=max_iid + 1, k_UI=K_UI, k_IL=K_IL)
test_model.to(PYTORCH_DEVICE)
test_model.load_state_dict(best_ncdg_model_state)

ndcg, hit = evaluate_model(test_model, test_dataloader, device=PYTORCH_DEVICE)

print(f"Test NDCG@{EVAL_K}: {ndcg:.4f}")
print(f"Test HIT@{EVAL_K}: {hit:.4f}")

Evaluating Validation:   0%|          | 0/79 [00:00<?, ?it/s]

Test NDCG@10: 0.6906
Test HIT@10: 0.7975
